# Customer Churn Prediction for a Telecom Provider
__Core Problem :__

Proactively identify customers at high risk of canceling their service, allowing the business to implement targeted retention strategies to reduce revenue loss.

## 01-Setup and Initial Exploration

In [ ]:
# Import necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
